 Fake News Detection with Explainability (BERT + SHAP + LIME)

This notebook builds a high-performance, interpretable Fake News classifier using BERT with SHAP and LIME for explainability.

In [ ]:
# 📦 Install required libraries
!pip install transformers shap lime datasets --quiet

In [ ]:
 Imports
import numpy as np
import pandas as pd
import shap
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset
from lime.lime_text import LimeTextExplainer

In [ ]:
# ✅ Device check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

In [ ]:
# 🔹 Load LIAR dataset sample
df = pd.read_csv('https://raw.githubusercontent.com/saurabhmathur96/LIAR-Dataset/main/train.tsv', sep='\t', header=None)
df = df[[2, 1]]
df.columns = ['text', 'label']
df['label'] = df['label'].replace({
    'false': 0, 'pants-fire': 0, 'barely-true': 0,
    'half-true': 1, 'mostly-true': 1, 'true': 1
})
df = df.sample(5000, random_state=42)

In [ ]:
# 📊 Train-Test Split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['text'].tolist(), df['label'].tolist(), test_size=0.15, stratify=df['label'], random_state=42)

In [ ]:
# 🧠 Tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_data(texts):
    return tokenizer(texts, truncation=True, padding=True, max_length=128)

train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)

train_dataset = Dataset.from_dict({**train_encodings, "label": train_labels})
val_dataset = Dataset.from_dict({**val_encodings, "label": val_labels})

In [ ]:
# 🤖 Load Model
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

In [ ]:
# 🛠️ Training Configuration
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=4,
    weight_decay=0.01,
    load_best_model_at_end=True,
    save_total_limit=1
)

In [ ]:
# 📦 Trainer Setup
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer)
)

# 🚀 Train
trainer.train()

In [ ]:
# 📈 Evaluate
preds = trainer.predict(val_dataset)
y_pred = np.argmax(preds.predictions, axis=1)
print("Classification Report:\n", classification_report(val_labels, y_pred))
print("Confusion Matrix:\n", confusion_matrix(val_labels, y_pred))